# Решение заданий к уроку 4

In [1]:
import matplotlib.pyplot as plt
import random

from matplotlib.colors import ListedColormap
from sklearn import datasets

import numpy as np

__1.__В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.).



__2.__Для задачи классификации обучить дерево решений с использованием критериев разбиения Джини и Энтропия. Сравнить качество классификации, сделать выводы. 



In [2]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction     

In [3]:
class DecisionTree:
    
    def __init__(self, min_leaf, criterion):
        self.min_leaf = min_leaf
        self.criterion = criterion
    
    # Расчет критерия

    def calc_criterion(self,labels):
        #  подсчет количества объектов разных классов
        classes = {}
        for label in labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1

        #  расчет критерия
        if self.criterion == 'gini':
            criter = 1
            for label in classes:
                p = classes[label] / len(labels)
                criter -= p ** 2
#            print('gini', criter)
        elif self.criterion == 'entropy':
            criter = 0
            for label in classes:
                p = classes[label] / len(labels)
                criter -= p * np.log2(p)
#            print('entropy',criter)
        return criter
        
    # Расчет качества
    def quality(self, true_labels, false_labels, criter):        
        # доля выбоки, ушедшая в левое поддерево
        p = float(true_labels.shape[0]) / (true_labels.shape[0] + false_labels.shape[0])
        return criter - p * self.calc_criterion(true_labels) - (1 - p) * self.calc_criterion(false_labels)         
        
    # Разбиение датасета в узле
    def split(self, data, labels, index, t):

        left = np.where(data[:, index] <= t)
        right = np.where(data[:, index] > t)

        true_data = data[left]
        false_data = data[right]
        true_labels = labels[left]
        false_labels = labels[right]

        return true_data, false_data, true_labels, false_labels 
        
    # Нахождение наилучшего разбиения
    def find_best_split(self, data, labels):

        criter = self.calc_criterion(labels)

        best_quality = 0
        best_t = None
        best_index = None
        
        n_features = data.shape[1]
    
        for index in range(n_features):
            # будем проверять только уникальные значения признака, исключая повторения
            t_values = np.unique([row[index] for row in data])

            for t in t_values:
                true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)
                #  пропускаем разбиения, в которых в узле остается менее 5 объектов
                if len(true_data) < self.min_leaf or len(false_data) < self.min_leaf:
                    continue

                current_quality = self.quality(true_labels, false_labels, criter)

                #  выбираем порог, на котором получается максимальный прирост качества
                if current_quality > best_quality:
                    best_quality, best_t, best_index = current_quality, t, index

        return best_quality, best_t, best_index
        
    # Построение дерева с помощью рекурсивной функции
    def build_tree(self, data, labels): 

        quality, t, index = self.find_best_split(data, labels)        

        #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
        if quality == 0:
            return Leaf(data, labels)

        true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)        

        # Рекурсивно строим два поддерева
        true_branch = self.build_tree(true_data, true_labels)
        false_branch = self.build_tree(false_data, false_labels)

        # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева        
        return Node(index, t, true_branch, false_branch)
    
    def classify_object(self, obj, node):
        #  Останавливаем рекурсию, если достигли листа
        if isinstance(node, Leaf):
            answer = node.prediction
            return answer

        if obj[node.index] <= node.t:
            return self.classify_object(obj, node.true_branch)
        else:
            return self.classify_object(obj, node.false_branch)

    def fit(self, data, labels):
        self.my_tree = self.build_tree(data, labels)
        
    def predict(self, data):        
        classes = []
        for obj in data:
            prediction = self.classify_object(obj, self.my_tree)        
            classes.append(prediction)
        return classes    

In [4]:
# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [20]:
# сгенерируем данные
classification_data, classification_labels = datasets.make_classification(n_samples=100,n_features = 2, n_informative = 2, 
                                                      n_classes = 2, n_redundant=0, 
                                                      n_clusters_per_class=1, random_state=2)

In [21]:
# Разобьем выборку на обучающую и тестовую

from sklearn import model_selection

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data, 
                                                                                     classification_labels, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

In [22]:
print(f'Критерий\t train_accuracy \t\t  test_accuracy')
for crit in ['gini','entropy']:
    tree = DecisionTree(min_leaf=5, criterion=crit)
    tree.fit(train_data, train_labels)
    train_answers = tree.predict(train_data)
    test_answers = tree.predict(test_data)
    print(f'{crit}\t\t {accuracy_metric(train_labels, train_answers)} \t\t {accuracy_metric(test_labels, test_answers)}')
    

Критерий	 train_accuracy 		  test_accuracy
gini		 98.57142857142858 		 86.66666666666667
entropy		 98.57142857142858 		 86.66666666666667


На небольших выборках Критерий Джини и Критерий Энтропии дают одинаковый результат, но на больших выборках уже могут давать разный результат но близкий

In [23]:
classification_data, classification_labels = datasets.make_classification(n_samples=1000,n_features = 2, n_informative = 2, 
                                                      n_classes = 2, n_redundant=0, 
                                                      n_clusters_per_class=1, random_state=2)
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data, 
                                                                                     classification_labels, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)
print(f'Критерий\t train_accuracy \t\t  test_accuracy')
for crit in ['gini','entropy']:
    tree = DecisionTree(min_leaf=5, criterion=crit)
    tree.fit(train_data, train_labels)
    train_answers = tree.predict(train_data)
    test_answers = tree.predict(test_data)
    print(f'{crit}\t\t {accuracy_metric(train_labels, train_answers)} \t\t {accuracy_metric(test_labels, test_answers)}')

Критерий	 train_accuracy 		  test_accuracy
gini		 94.71428571428572 		 88.66666666666667
entropy		 94.85714285714286 		 90.0


__3*.__ Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.